In [ ]:
!pip install orjson -q
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast, os, re, json, requests, orjson
from glob import glob
from time import sleep
from bs4 import BeautifulSoup
data_folder = ''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.0 MB/s eta 0:00:00


---
### make onsides data

In [9]:
ade_text_table_df = pd.read_csv(data_folder+'data/ade_text_table_v0924.csv')
#to run the extraction, remove the tabulated data if it is in the text.
ade_text_table_df['ade_txt'] = ade_text_table_df.ade_text.apply(lambda x: x.split('Tabulated')[0])
ade_text_table_df.head(1)

,drug,ade_text,ade_txt
0,extavia,e ffects Summary of the safety profile At the...,e ffects Summary of the safety profile At the...


In [10]:
#read in meddra database
externals = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/external_data/'
umls_map = pd.read_csv(externals+'umls_meddra_en.csv')
umls_map['STR'] = umls_map.STR.apply(lambda x: x.lower())
umls_map['LEN'] = umls_map.STR.apply(lambda x: len(x))
umls_map = umls_map[umls_map.LEN >= 5][['STR', 'TTY', 'SDUI']].drop_duplicates(subset='STR')
umls_map_dict = dict(zip(umls_map.STR, umls_map.SDUI))
umls_map.head(1)

,STR,TTY,SDUI
0,acute abdomen,PT,10000647


In [11]:
exact_terms = []
for i, row in tqdm(ade_text_table_df.iterrows()):
  label_id = row['drug']
  text = row['ade_txt'].lower()
  found_terms = list()
  for mdr_term in umls_map_dict.keys():
    if text.find(mdr_term) == -1:
      continue
    else:
      li = text.split(mdr_term)
      start_pos = 0
      for i in range(len(li)-1):
          # the occurrence of the word is at the end of the previous string
          start_pos = sum([len(li[j]) for j in range(i+1)]) + i*len(mdr_term)
          if not mdr_term == text[start_pos:(start_pos+len(mdr_term))]:
              raise Exception(f" mdr_term: '{mdr_term}', term_in_text: '{text[start_pos:(start_pos+len(mdr_term))]}'")
          found_terms.append((mdr_term, umls_map_dict[mdr_term], start_pos, len(mdr_term)))
  exact_terms.append([label_id, found_terms])

1512it [02:44,  9.19it/s]


In [12]:
exact_terms_df = pd.DataFrame(exact_terms, columns=['label_id', 'found_terms'])
exact_terms_df = exact_terms_df.explode('found_terms')
exact_terms_df['len'] = exact_terms_df['found_terms'].apply(lambda x: x[3] if str(x) != 'nan' else None)
exact_terms_df = exact_terms_df[exact_terms_df['len'] >= 5]
exact_terms_df['found_term'] = exact_terms_df['found_terms'].apply(lambda x: x[0] if str(x) != 'nan' else None)
exact_terms_df['meddra_id'] = exact_terms_df['found_terms'].apply(lambda x: x[1] if str(x) != 'nan' else None)
exact_terms_df['location'] = exact_terms_df['found_terms'].apply(lambda x: x[2] if str(x) != 'nan' else None)
exact_terms_df = exact_terms_df.drop(['found_terms', 'len'], axis = 1)
exact_terms_df.head(1)

<ipython-input-12-448ea2f8116a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exact_terms_df['found_term'] = exact_terms_df['found_terms'].apply(lambda x: x[0] if str(x) != 'nan' else None)
<ipython-input-12-448ea2f8116a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exact_terms_df['meddra_id'] = exact_terms_df['found_terms'].apply(lambda x: x[1] if str(x) != 'nan' else None)


,label_id,found_term,meddra_id,location
0,extavia,anaemia,10002034,785


In [13]:
building_strings = []
ade_text_table_dict = dict(zip(ade_text_table_df.drug, ade_text_table_df.ade_txt))
for i, row in tqdm(exact_terms_df.iterrows()):
  term, label_id, start_pos = row['found_term'], row['label_id'], row['location']
  #default settings
  nwords, prop_before = 125, 0.125
  #pull the full text
  ar_text = ade_text_table_dict[label_id]

  term_nwords = len(term.split())
  size_before = max(int((nwords-2*term_nwords)*prop_before), 1)
  size_after = max(int((nwords-2*term_nwords)*(1-prop_before)), 1)

  before_text = ar_text[:start_pos]
  after_text = ar_text[(start_pos+term_nwords):]

  before_parts = before_text.split()[-1*size_before:]
  after_parts = after_text.split()[:size_after]

  li = [term]
  li.extend(before_parts)
  li.append('EVENT')
  li.extend(after_parts)
  example_string = ' '.join(li)
  building_strings.append(example_string)
exact_terms_df['string'] = building_strings

65468it [00:06, 9533.41it/s] 


In [14]:
exact_terms_df.to_csv(data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv', index=False)
exact_terms_df.head(1)

,label_id,found_term,meddra_id,location,string
0,extavia,anaemia,10002034,785,anaemia IU Extavia treatment. The most serious...


---
### format dataframe for onsides model

In [15]:
#required columns : section, drug, label_id, set_id, spl_version, pt_meddra_id, pt_meddra_term
exact_terms_df = pd.read_csv(data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv')
exact_terms_df['section'] = 'AR'
exact_terms_df['set_id'] = exact_terms_df['label_id']
exact_terms_df.head(1)

,label_id,found_term,meddra_id,location,string,section,set_id
0,extavia,anaemia,10002034,785,anaemia IU Extavia treatment. The most serious...,AR,extavia


In [16]:
folder = '/content/drive/MyDrive/pop_pharmacogenomics/fda/onsides-2.0.0/data/'
drug_map = pd.read_csv(folder+'spl/maps/20230512/rxnorm_mappings.txt', delimiter = '|')
drug_id_dict = dict(zip(drug_map.SETID, drug_map.RXCUI))
drug_ver_dict = dict(zip(drug_map.SETID, drug_map.SPL_VERSION))
exact_terms_df['drug'] = exact_terms_df.set_id.apply(lambda x: drug_id_dict[x] if x in drug_id_dict.keys() else None)
exact_terms_df['spl_version'] = exact_terms_df.set_id.apply(lambda x: drug_ver_dict[x] if x in drug_ver_dict.keys() else None)

llt_pt = pd.read_csv(folder+'meddra_llt_pt_map.txt', delimiter = '|')
llt_pt_id_dict = dict(zip(llt_pt.llt_concept_code, llt_pt.pt_concept_code))
llt_pt_term_dict = dict(zip(llt_pt.llt_concept_code, llt_pt.pt_concept_name))
exact_terms_df['pt_meddra_id'] = exact_terms_df.meddra_id.apply(lambda x: llt_pt_id_dict[x] if x in llt_pt_id_dict.keys() else None)
exact_terms_df['pt_meddra_term'] =  exact_terms_df.meddra_id.apply(lambda x: llt_pt_term_dict[x] if x in llt_pt_term_dict.keys() else None)

exact_terms_df.to_csv(data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv', index=False)
exact_terms_df.head(1)

,label_id,found_term,meddra_id,location,string,section,set_id,drug,spl_version,pt_meddra_id,pt_meddra_term
0,extavia,anaemia,10002034,785,anaemia IU Extavia treatment. The most serious...,AR,extavia,None,None,10002034.0,Anaemia


In [ ]:
exact_terms_df = pd.read_csv(data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv')
epa = pd.read_excel(data_folder+'data/Medicines_output_european_public_assessment_reports.xlsx', skiprows = 8)
epa['Medicine name'] = epa['Medicine name'].apply(lambda x: x.lower())
epa['Active substance'] = epa['Active substance'].apply(lambda x: x.lower() if str(x) != 'nan' else None)
epa = epa[epa['Medicine name'].isin(exact_terms_df.label_id)][['Medicine name', 'Active substance', 'Revision number']]
externals = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/external_data/'
umls_rxnorm = pd.read_csv(externals+'umls_rxnorm.csv')
umls_rxnorm_dict = dict(zip(umls_rxnorm.STR, umls_rxnorm.CODE))
epa['rxnorm_code'] = epa['Active substance'].apply(lambda x: umls_rxnorm_dict[x] if x in umls_rxnorm_dict.keys() else None)
epa_dict = dict(zip(epa['Medicine name'], epa['rxnorm_code']))
epa_ver_dict = dict(zip(epa['Medicine name'], epa['Revision number']))
exact_terms_df['drug'] = exact_terms_df.apply(lambda x: epa_dict[x.label_id] if str(x.drug) != 'nan' else x.drug, axis = 1)
exact_terms_df['spl_version'] = exact_terms_df.apply(lambda x: epa_ver_dict[x.label_id] if str(x.spl_version) != 'nan' else x.spl_version, axis = 1)

In [ ]:
exact_terms_df.to_csv(data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv', index=False)
exact_terms_df.head(1)

,label_id,found_term,meddra_id,location,string,section,set_id,drug,spl_version,pt_meddra_id,pt_meddra_term
0,extavia,anaemia,10002034,785,anaemia IU Extavia treatment. The most serious...,AR,extavia,NaN,NaN,10002034.0,Anaemia


### run onsides model

In [ ]:
#prep onsides materials
folder = ''
ar_model = folder + '/bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth'
bw_model = folder + '/bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth'
import os
os.chdir('') #change to path that you pull the repo to
!python3 -m pip install -r requirements.txt

In [ ]:
#use onsides model
f = data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv'
!python3 src/predict.py --model $ar_model --examples $f

2023-09-24 21:08:34.202246: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 21:08:34.270655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 21:08:35.407156: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading model from bestepoch-bydrug-PMB_14-AR-125-all_222_24_25_2.5e-05_256_32.pth
Model
-------------------
 prefix: bestepoch-bydrug-PMB
 network: PMB
 refset: 14
 refsource: all
 refsection: AR
 refnwords: 125
 np_random_seed: 222
 split_method: 2

In [20]:
res = data_folder+'data/bestepoch-bydrug-PMB-sentences-rx_ref14-AR-125-all_222_24_25_2.5e-05_256_32.csv.gz'
ex = data_folder+'data/sentences-rx_method14_nwords125_clinical_bert_application_set_AR_v0924.csv'
threshold = 0.463
res = pd.read_csv(res, header=None, names=['Pred0', 'Pred1'])
ex = pd.read_csv(ex)
df = pd.concat([ex, res], axis=1)
print(df.shape[0])
df = df[df.Pred0 > threshold]
print(df.shape[0])
df.head()

65468
47906


,label_id,found_term,meddra_id,location,string,section,set_id,drug,spl_version,pt_meddra_id,pt_meddra_term,Pred0,Pred1
0,extavia,anaemia,10002034,785,anaemia IU Extavia treatment. The most serious...,AR,extavia,NaN,NaN,10002034.0,Anaemia,1.035913,0.0
1,extavia,haemolytic anaemia,10018916,774,haemolytic anaemia million IU Extavia treatmen...,AR,extavia,NaN,NaN,10018916.0,Haemolytic anaemia,1.885699,0.0
3,extavia,infec,10021881,556,infec site reactions occurred frequently after...,AR,extavia,NaN,NaN,NaN,NaN,2.371783,0.0
4,extavia,fever,10037660,229,fever with further treatment. The most frequen...,AR,extavia,NaN,NaN,10037660.0,Pyrexia,1.272035,0.0
5,extavia,headache,10019211,275,headache frequently observed adverse reactions...,AR,extavia,NaN,NaN,10019211.0,Headache,1.979937,0.0


In [24]:
df.to_csv(data_folder+'data/ade_text_table_onsides_pred_v0924.csv', index=False)

In [ ]:
#build tables
r = data_folder+'data/bestepoch-bydrug-PMB-sentences-rx_ref14-AR-125-all_222_24_25_2.5e-05_256_32.csv.gz'
!python3 src/create_onsides_datafiles.py --release v2.0.0-AR --results $r --examples $f

 prefix: bestepoch-bydrug-PMB-sentences-rx
 refset: ref14-AR-125-all
 np_random_seed: 222
 split_method: 24
 EPOCHS: 25
 LR: 2.5e-05
 threshold: 0.4633
 max_length: 2.5e-05
 batch-size: 32.csv
 compiled file: /content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/eu_drug_label/data/compiled/v2.0.0/AR.csv.gz
Loading results file...
 res.shape: (65468, 2)
Loding examples file...
 ex.shape: (65468, 11)
Concatenating results file to examples file...
Grouping predictions by drug label and adverse event term, and taking the mean prediction score...
/content/drive/MyDrive/pop_pharmacogenomics/fda/onsides-2.0.0/src/create_onsides_datafiles.py:97: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = df.groupby(by=['section', 'drug', 'label_id', 'set_id', 'spl_version', 'pt_meddra_id', 'pt_meddra_t